In [4]:
from google.colab import files
uploaded = files.upload()


Saving Siddharth_Associates_sample data 2.xlsx to Siddharth_Associates_sample data 2.xlsx


In [5]:
import os
os.listdir()


['.config', 'Siddharth_Associates_sample data 2.xlsx', 'sample_data']

In [6]:
df = pd.read_excel("Siddharth_Associates_sample data 2.xlsx")


In [7]:
import pandas as pd
import re

# Load sample data
df = pd.read_excel("Siddharth_Associates_sample data 2.xlsx")

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# ------------------------------
# 1. GOODS DESCRIPTION CLEANING
# ------------------------------
df['goods_description'] = df['goods_description'].str.lower().str.strip()

# 1A. Extract Model Name
def get_model(text):
    match = re.search(r"(model\s*\w+)", text)
    return match.group(0) if match else None

df["model_name"] = df["goods_description"].apply(get_model)

# 1B. Extract Model Number
def get_model_number(text):
    match = re.search(r"\b[A-Za-z]*\d{2,}[A-Za-z0-9]*\b", text)
    return match.group(0) if match else None

df["model_number"] = df["goods_description"].apply(get_model_number)

# 1C. Extract Material Type (steel, glass, etc.)
materials = ["steel", "glass", "plastic", "copper"]

def get_material(text):
    for m in materials:
        if m in text:
            return m
    return None

df["material_type"] = df["goods_description"].apply(get_material)

# 1D. Extract Capacity
def get_capacity(text):
    match = re.search(r"(\d+ ?l|\d+ ?ml|\d+ ?kg)", text)
    return match.group(0) if match else None

df["capacity"] = df["goods_description"].apply(get_capacity)

# 1E. Extract Embedded Quantity
def get_emb_qty(text):
    match = re.search(r"(\d+ ?pcs|\d+ ?nos|\d+ ?pieces)", text)
    return match.group(0) if match else None

df["embedded_quantity"] = df["goods_description"].apply(get_emb_qty)

# ------------------------------
# 2. STANDARDIZE UNIT
# ------------------------------
if "unit" in df.columns:
    df["unit"] = df["unit"].str.lower().replace({
        "pieces": "pcs",
        "piece": "pcs",
        "nos": "pcs"
    })

df.to_csv("cleaned_trade_data.csv", index=False)
print("Parsing & cleaning completed.")


Parsing & cleaning completed.


In [8]:
import os
os.listdir()


['.config',
 'cleaned_trade_data.csv',
 'Siddharth_Associates_sample data 2.xlsx',
 'sample_data']

In [9]:
import pandas as pd

df = pd.read_csv("cleaned_trade_data.csv")

df["grand_total_inr"] = df["total_value_inr"] + df["duty_paid_inr"]

def map_category(hsn):
    hsn = str(hsn)
    if hsn.startswith("73"):
        return "Steel"
    elif hsn.startswith("70"):
        return "Glass"
    elif hsn.startswith("82"):
        return "Cutlery"
    else:
        return "Others"

df["category"] = df["hs_code"].apply(map_category)

def map_subcat(text):
    if "scrub" in text:
        return "Scrubber"
    if "strainer" in text:
        return "Tea Strainer"
    return "Others"

df["sub_category"] = df["goods_description"].apply(map_subcat)

df["landed_cost_per_unit"] = df["grand_total_inr"] / df["qty"]

df.to_csv("final_feature_engineered_data.csv", index=False)

print("Feature engineering completed.")


Feature engineering completed.


In [10]:
from google.colab import files
files.download("final_feature_engineered_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
import os
os.listdir()


['.config',
 'cleaned_trade_data.csv',
 'final_feature_engineered_data.csv',
 'Siddharth_Associates_sample data 2.xlsx',
 'sample_data']